##📌 SCRAPING BOOKING ATRACCIONES
Este notebook combina datos de múltiples archivos CSV de Booking para obtener URLs de atracciones turísticas. Luego, utiliza Selenium para visitar cada URL, extraer las reseñas de los usuarios (incluyendo score, texto, país, categoría y fecha) y guarda todos los comentarios recopilados en un nuevo archivo CSV.

# 📥 Imports y librerías

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import glob

# ⚙️ Configuración

In [ ]:
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Quita esto si quieres ver el navegador
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(), options=chrome_options)
actions = ActionChains(driver)

# 📝 Bloque de código

In [ ]:

# Lista de archivos CSV a procesar
csv_files = ["booking_barcelona_actualizado.csv", "booking_grancanaria_actualizado.csv", "booking_madrid_actualizado.csv",
             "booking_malaga_actualizado.csv", "booking_mallorca_actualizado.csv", "booking_sevilla_actualizado.csv",
             "booking_tenerife_actualizado.csv", "booking_valencia_actualizado.csv"] # Agrega aquí los nombres de tus archivos

# DataFrame vacío para almacenar todos los datos
all_df = pd.DataFrame()

# Leer y concatenar cada archivo CSV
for file in csv_files:
    try:
        df = pd.read_csv(file)
        all_df = pd.concat([all_df, df], ignore_index=True)
    except FileNotFoundError:
        print(f"Error: El archivo {file} no fue encontrado.")
    except Exception as e:
        print(f"Error al leer el archivo {file}: {e}")

df = all_df # Usamos 'df' como el DataFrame principal para mantener la compatibilidad con el resto del código

In [ ]:
comentarios_df = pd.DataFrame(columns=["titulo", "review_score", "comentario", "pais", "categoria", "fecha"])

# 📝 Funciones usadas

Este notebook utiliza las siguientes funciones definidas localmente:

*   **`scroll_lentamente(pixeles, pausas, repeticiones)`**: Realiza un scroll suave en la página web una cantidad específica de veces para asegurar que el contenido dinámico se cargue correctamente. Permite configurar la cantidad de píxeles a desplazar, la pausa entre scrolls y el número de repeticiones.
*   **`scrapear_experiencia(url)`**: Función principal para extraer reseñas de una URL de Booking. Navega a la página, maneja pop-ups de cookies, extrae el título, busca y hace click para ver comentarios, itera a través de las reseñas extrayendo score, texto, país, categoría y fecha, y maneja la paginación dentro del modal de reseñas. Retorna una lista de diccionarios con los datos de las reseñas.

# 🌐 Configuración de Selenium / WebDriver

In [ ]:
def scroll_lentamente(pixeles=300, pausas=0.5, repeticiones=5):
    for _ in range(repeticiones):
        driver.execute_script(f"window.scrollBy(0, {pixeles});")
        time.sleep(pausas)

# 🌐 Configuración de Selenium / WebDriver

In [ ]:
def scrapear_experiencia(url):
    comentarios = []

    try:
        driver.get(url)
        time.sleep(1)

        # Cerrar cookies si aparece
        try:
            aceptar_cookies = driver.find_element(By.ID, "onetrust-accept-btn-handler")
            aceptar_cookies.click()
            time.sleep(1)
        except:
            pass

        # Scroll para que cargue el botón
        scroll_lentamente(pixeles=300, pausas=0.2, repeticiones=2)

        # Extraer título
        try:
            titulo = driver.find_element(By.CLASS_NAME, "a4ac75716e.css-1mpsda3").text
        except:
            titulo = "Sin título"

        # Ver y hacer click en botón de comentarios
        try:
            ver_comentarios_btn = driver.find_element(By.CSS_SELECTOR, '[data-testid="reviews-link"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", ver_comentarios_btn)
            time.sleep(1)
            ver_comentarios_btn.click()
            time.sleep(1)
        except:
            print(f"Botón 'ver todos los comentarios' no disponible en: {url}")
            return []

        # Esperar que cargue el modal
        try:
            review_modal = driver.find_element(By.CLASS_NAME, "a9f1d9ba2c")
        except:
            print("No se encontró el modal de reseñas.")
            return []

        # Loop para recorrer páginas
        while True:
            review_items = review_modal.find_elements(By.CSS_SELECTOR, '[data-testid="review-item"]')
            for item in review_items:
                try:
                    score_raw = item.find_element(By.CSS_SELECTOR, '[data-testid="review-star-rating"]')
                    score_text = score_raw.get_attribute("aria-label")
                    score = int(score_text.strip()[0])
                except:
                    score = None

                try:
                    texto = item.find_element(By.CLASS_NAME, "b99b6ef58f").text.strip()
                except:
                    texto = ""

                pais = ""
                categoria = ""
                fecha = ""

                try:
                    contenedor = item.find_element(By.CSS_SELECTOR, 'div.css-7jm4mj')
                    campos = contenedor.find_elements(By.CSS_SELECTOR, 'div.fff1944c52.fb14de7f14')

                    if len(campos) > 0:
                        pais = campos[0].text.strip()
                    if len(campos) > 1:
                        categoria = campos[1].text.strip()
                    if len(campos) > 2:
                        fecha = campos[2].text.strip()
                        # Limpieza si incluye el texto completo
                        fecha = fecha.replace("Publicado el ", "").replace(" en Booking.com", "").strip()

                except Exception as e:
                    print("Error extrayendo país/categoría/fecha:", e)

                # Agregar al diccionario final
                comentarios.append({
                    "titulo":     titulo,
                    "review_score": score,
                    "comentario": texto,
                    "pais":       pais,
                    "categoria":  categoria,
                    "fecha":      fecha
                })

            # Botón "Siguiente"
            try:
                pagination_container = review_modal.find_element(By.CSS_SELECTOR, '[data-testid="reviews-pagination"]')
                next_btn = pagination_container.find_element(By.CSS_SELECTOR, 'button[aria-label="Siguiente"]')

                if next_btn.get_attribute("disabled") is not None:
                    print("Fin del paginado.")
                    break

                # Scroll dentro del modal
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_btn)
                time.sleep(1)
                driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
                time.sleep(1)

                try:
                    next_btn.click()
                    time.sleep(1)
                except ElementClickInterceptedException:
                    break

            except NoSuchElementException:
                print("No se encontró el botón 'Siguiente' dentro de la paginación.")
                break

    except Exception as e:
        print(f"Error general en {url}: {e}")

    return comentarios

# 📝 Extracción de reseñas

Este código itera sobre cada fila de un DataFrame, extrayendo la URL de cada una para raspar (scrapear) los comentarios de cada URL usando la función scrapear_experiencia. Los comentarios obtenidos se van agregando a otro DataFrame. Finalmente, guarda todos los comentarios recopilados en un archivo CSV y cierra el navegador de Selenium.

In [ ]:
for idx, row in df.iterrows():
    print(f"Procesando {idx + 1}/{len(df)}: {row['url']}")
    resultados = scrapear_experiencia(row["url"])
    if resultados:
        comentarios_df = pd.concat([comentarios_df, pd.DataFrame(resultados)], ignore_index=True)
    time.sleep(1)

# Guardar resultados
comentarios_df.to_csv("booking_barcelona_comentarios_completos.csv", index=False)
print("Archivo final guardado con éxito.")

# Cerrar navegador
driver.quit()